In [ ]:
import requests
from bs4 import BeautifulSoup
from pexpect import searcher_string
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import matplotlib.pyplot as plt
import numpy as np
import time
from matplotlib import rc
import re
import csv
import sys

In [ ]:
keyword = "등산여행"

url = "https://www.instagram.com/explore/tags/{}/".format(keyword)

instagram_tags = []
instagram_tag_dates = []

driver = wd.Chrome("./chromedriver")
driver.get(url)
time.sleep(3)

# 로그인 하기
# /html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button
# document.querySelector("#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.ctQZg > div > span > a:nth-child(1) > button")
# <button class="sqdOP  L3NKy   y3zKF     " type="button">로그인</button>
#
# // *[ @ id = "react-root"] / section / nav / div[2] / div / div / div[3] / div / span / a[1] / button

login_section = '//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button'
driver.find_element_by_xpath(login_section).click()
time.sleep(2)


elem_login = driver.find_element_by_name("username")
elem_login.clear()
elem_login.send_keys('01064747059')

elem_login = driver.find_element_by_name('password')
elem_login.clear()
elem_login.send_keys('vmfhwprxm')

time.sleep(1)

xpath = """//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button"""
driver.find_element_by_xpath(xpath).click()

time.sleep(4)


good_xpath = """//*[@id="react-root"]/section/main/div/div/div/section/div/button"""
driver.find_element_by_xpath(good_xpath).click()

time.sleep(4)

driver.get(url)
time.sleep(3)

In [ ]:
# 게시물 개수 정보를 가져온다
totalCount = driver.find_element_by_class_name("g47SY ").text
totalCount = totalCount.replace(",","")

driver.find_element_by_css_selector('div.v1Nh3.kIKUG._bz0w').click()

for i in range(int(totalCount)):
    time.sleep(1)
    try:
        data = driver.find_element_by_css_selector('.XQXOT') #게시물과 댓글
        tag_raw = data.text
        postList = tag_raw.split("\n")
#         print(postList)
        for i in postList:
            if re.match('[0-9]*w$', i):
                idx=postList.index(i)
#                 print(postList.index(i))
                break
#         print(postList[idx:])

        tags = re.findall('#[A-Za-z0-9가-힣]+', tag_raw)
        tag = ''.join(tags).replace("#", " ")

        tag_data = tag.split()

        for tag_one in tag_data:
            instagram_tags.append(tag_one)
        print(instagram_tags)

        date = driver.find_element_by_css_selector("time._1o9PC.Nzb55").text  # 날짜 선택
        date = date[:-1]
        print(date)
        if date.find('시간') != -1 or date.find('일') != -1 or date.find('분') != -1:
            instagram_tag_dates.append('0주')
        else:
            if int(date)<=8:
                instagram_tag_dates.append(date)

    except:
        instagram_tags.append("error")
        instagram_tag_dates.append('error')
    try:
        WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a._65Bje.coreSpriteRightPaginationArrow')))
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()

    except:
        driver.close()
    time.sleep(2)

instagram_date_dic={}
for i in range(len(instagram_tag_dates)):
    if int(instagram_tag_dates[i]) in instagram_date_dic:
        instagram_date_dic[int(instagram_tag_dates[i])]+=1
    else:
        instagram_date_dic[int(instagram_tag_dates[i])]=1
# print(instagram_tag_dates)
# print(instagram_tags)